<a href="https://colab.research.google.com/github/ansonkwokth/TableTennisPrediction/blob/dev/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ansonkwokth/TableTennisPrediction.git
%cd TableTennisPrediction

Cloning into 'TableTennisPrediction'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 289 (delta 78), reused 15 (delta 15), pack-reused 161 (from 1)
Receiving objects: 100% (289/289), 3.93 MiB | 9.19 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/TableTennisPrediction


In [2]:

import pandas as pd
from tabulate import tabulate
from utils import data_loader as dl

import numpy as np
from model.Elo import Elo
from model.ModifiedElo import ModifiedElo
from model.ensemble import BaggingRatingSystem

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import copy

import warnings
warnings.filterwarnings('ignore')

In [20]:
GAME = 'TTStar'
# GAME = 'TTCup'
# GAME = 'SetkaCup'
# GAME = 'SetkaCupWomen'
# GAME = 'LigaPro'


In [21]:
match GAME:
    case 'TTStar':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'TTCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCupWomen':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'LigaPro':
        years = [2022, 2023, 2024]
    case _:
        raise ValueError("Invalid game selected.")


text_data_game = dl.load_game_data(GAME, years, '../')
text_data = {
    year: text_data_game[year] for year in years
}
df = dl.create_game_dfs(GAME, years, text_data)

Loading ..//TTStar2020.txt
Loading ..//TTStar2021.txt
Loading ..//TTStar2022.txt
Loading ..//TTStar2023.txt
Loading ..//TTStar2024.txt


In [22]:
# Generate ID indices for each pair of rows in the DataFrame
idx_lt = [i for i in range(len(df) // 2) for _ in range(2)]
df['ID'] = idx_lt  # Assign to the 'ID' column

# Reset the DataFrame index to ensure it's sequential
df.reset_index(drop=True, inplace=True)

# Get unique players and store them in player_lt
player_lt = df['Player'].unique()



In [23]:
def format_to_array(df: pd.DataFrame) -> np.ndarray:

    # info_col = ['ID', 'Round', 'Datetime', 'Game', 'Date', 'Time']
    info_col = ['Round', 'Datetime', 'Game', 'Date', 'Time']
    col = [item for item in df.columns if item not in info_col]

    df[[c for c in col if "Set" in c]] = df[[c for c in col if "Set" in c]].astype(float)
    X = df[col].values.reshape(-1, 2, len(col))
    return X

In [24]:
X_all = format_to_array(df)

In [25]:
modelMElo = ModifiedElo()
for _ in range(5):
    modelMElo.fit(X_all)


Training model: 100%|██████████| 17257/17257 [00:00<00:00, 19960.16it/s]


# Print

In [26]:
def find_player(name):
    p_lt = []
    for k, v in modelMElo.params.items():
        if name in k:
            print(k)
            p_lt.append(k)
    return p_lt

In [27]:
def find_OU(player1, player2):
    set1_dt = modelMElo.predict_set_config(player1, player2)[-1]
    set2_dt = modelMElo.predict_set_config(player2, player1)[-1]

    set_sum_dt = {}
    for k1, v1 in set1_dt.items():
        v2 = set2_dt[k1]
        set_sum_dt[k1] = v1 + v2



    p_O = 0
    p_U = 0
    for k, v in set_sum_dt.items():
        if k > line:
            p_O += v
        else:
            p_U += v
    return p_O, p_U

In [28]:
df_lt = []

In [197]:
p1_lt = find_player("Kei")
print("-"*30)
p2_lt = find_player("Lin")


Keinath T.
------------------------------
Lindsoe D.


In [198]:
player1 = p1_lt[0]
player2 = p2_lt[0]
player1, player2

('Keinath T.', 'Lindsoe D.')

In [199]:
odd1 = 1.1
odd2 = 6.5

In [200]:
time = "22:30"

date = "Feb 8"


line = 18.5


found_p1, found_p2, p_game = modelMElo.predict_game(player1, player2)
found_p1, found_p2, p_set = modelMElo.predict_set(player1, player2)
p_O, p_U = find_OU(player1, player2)
winner = player1 if p_game > 0.5 else player2
p_set_win = p_set if winner == player1 else 1 - p_set
p_game_win = p_game if winner == player1 else 1 - p_game
p_OU = p_O if p_O > p_U else p_U
OU = "O" if p_O > p_U else "U"

winnerOdd = odd1 if winner == player1 else odd2
loserOdd = odd2 if winner == player1 else odd1

In [201]:
if (found_p1 and found_p2):
    df_row = pd.DataFrame({
                         "Time": [time],
                         "Player1": [player1], "Player2": [player2], "Winner": [winner],
                         "Set": [f"{p_set_win*100:.1f}%"], "Game": [f"{p_game_win*100:.1f}%"],
                         "pGame": [p_game_win],
                         "WinnerOdd": [winnerOdd], "LoserOdd": [loserOdd]
                         })
                        #  "O/U": [f"{OU} {line}: {p_OU*100:.1f}%"]})

    print(tabulate(df_row, headers = 'keys', tablefmt = 'psql', showindex=False))


+--------+------------+------------+------------+-------+--------+----------+-------------+------------+
| Time   | Player1    | Player2    | Winner     | Set   | Game   |    pGame |   WinnerOdd |   LoserOdd |
|--------+------------+------------+------------+-------+--------+----------+-------------+------------|
| 22:30  | Keinath T. | Lindsoe D. | Keinath T. | 89.7% | 99.1%  | 0.990577 |         1.1 |        6.5 |
+--------+------------+------------+------------+-------+--------+----------+-------------+------------+


In [202]:
p_U, p_O

(0.6926051748208125, 0.30739482517918776)

In [203]:
df_lt.append(df_row)

In [204]:
df_print = pd.concat(df_lt)
df_print.index = range(len(df_lt))
print(tabulate(df_print, headers = 'keys', tablefmt = 'psql', showindex=False))
# print(tabulate(df_print, headers = 'keys', tablefmt = 'psql'))

+--------+-------------+-------------+-------------+-------+--------+----------+-------------+------------+
| Time   | Player1     | Player2     | Winner      | Set   | Game   |    pGame |   WinnerOdd |   LoserOdd |
|--------+-------------+-------------+-------------+-------+--------+----------+-------------+------------|
| 09:00  | Stalzer A.  | Moravek R.  | Moravek R.  | 56.8% | 62.6%  | 0.626047 |       1.53  |       2.37 |
| 09:30  | Koldas T.   | Martinko T. | Martinko T. | 56.0% | 61.2%  | 0.611966 |       1.83  |       1.83 |
| 11:00  | Stalzer A.  | Martinko T. | Martinko T. | 73.9% | 88.4%  | 0.884449 |       1.28  |       3.5  |
| 11:30  | Koldas T.   | Kadlec V.   | Koldas T.   | 60.4% | 68.9%  | 0.689286 |       1.83  |       1.83 |
| 12:00  | Skala R.    | Moravek R.  | Moravek R.  | 50.3% | 50.5%  | 0.505224 |       1.33  |       3.25 |
| 13:00  | Stalzer A.  | Kadlec V.   | Kadlec V.   | 59.5% | 67.3%  | 0.673192 |       1.66  |       2.1  |
| 13:30  | Skala R.    | Mar

In [205]:


df_print.to_csv("../df_print.csv")